In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

In [2]:
# Utils
start_date = '2024-06-20 00:00:00'
end_date = '2024-07-20 23:59:59'

def inRange(start_date, end_date, time):
    return start_date <= time <= end_date

before_start=True

In [3]:
test = '2024-06-19 23:59:59'
inRange(start_date, end_date, test)

False

In [4]:
# 게시물 검색 페이지 (page_num으로 지정 가능)
driver = webdriver.Chrome() 
page_num=0
driver.get(f'https://www.clien.net/service/search?q=%EB%A5%B4%EB%85%B8&sort=recency&p={page_num}&boardCd=&isBoard=false')
time.sleep(3)

- 검색했을 때, 최대 50페이지만 보여줌

In [5]:
news_titles = driver.find_elements(By.CLASS_NAME, "subject_fixed")
datetimes = driver.find_elements(By.CLASS_NAME, "timestamp")
nicknames = driver.find_elements(By.CLASS_NAME, "nickname")

titles = []
hrefs = []
created_ats = []

for i in news_titles:
    titles.append(i.text)
    hrefs.append(i.get_attribute('href'))

for i in datetimes:
    created_ats.append(i.get_attribute('textContent'))

In [6]:
authors = []
image_elements = driver.find_elements(By.CSS_SELECTOR, 'span.nickname img')
gif_nick_cnt = 0
for idx, elem in enumerate(nicknames):
    nickname = elem.text
    if nickname == '':
        nickname = image_elements[gif_nick_cnt].get_attribute('alt')
        gif_nick_cnt+=1
    authors.append(nickname)
print(authors)

['fiat', '정의로운국가', '요트맨', 'startspark', 'WhoIsIt', 'DMWriter', 'eeeeee', '요트맨', '이게무슨개소리야', '요트맨', '마지막변명', '펀드라이빙', '이게무슨개소리야', 'Rize', '매복', '야광멧돼지', 'Serafim', '코스믹', 'ㅇㄷㄷ', '파랑덕후']


In [7]:
print(len(nicknames))

20


In [8]:
for href in hrefs:
    comments_nicknames = []
    # open new page
    driver.get(href)
    time.sleep(3)
    class_name = 'post_article' 

    # get contents
    # paragraphs = driver.find_elements(By.CSS_SELECTOR, f'.{class_name} p, .{class_name} h1, \
    # .{class_name} h2, .{class_name} h3, .{class_name} h4, .{class_name} h5, .{class_name} h6')

    # content=""
    # for para in paragraphs:
    #     content += para.text

    # get comments
    comment_ad_banner = driver.find_element(By.CLASS_NAME, "comment ad_banner")
    nicknames = comment_ad_banner.find_elements(By.CLASS_NAME, "nickname")
    for idx, elem in enumerate(nicknames):
        nickname = elem.text
        comments_nicknames.append(nickname)

    print(comments_nicknames)
    break

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".comment ad_banner"}
  (Session info: chrome=127.0.6533.88); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001005a5088 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x000000010059d764 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x00000001001ac82c cxxbridge1$string$len + 88524
3   chromedriver                        0x00000001001f0834 cxxbridge1$string$len + 367060
4   chromedriver                        0x000000010022848c cxxbridge1$string$len + 595500
5   chromedriver                        0x00000001001e5474 cxxbridge1$string$len + 321044
6   chromedriver                        0x00000001001e60e4 cxxbridge1$string$len + 324228
7   chromedriver                        0x00

In [ ]:
driver.quit()